## Computer Vision PA #2: Structure from Motion (SfM)
20195003 고강빈

In [98]:
import numpy as np
import cv2 as cv

In [99]:
img1 = cv.imread('./SfM/Data/sfm03.jpg')
img2 = cv.imread('./SfM/Data/sfm04.jpg')

# create SIFT instance
sift = cv.SIFT_create()
# sift = cv.xfeatures2d.SIFT_create()

# detect and compute keypoints
img1_kp, img1_des = sift.detectAndCompute(img1, None)
img2_kp, img2_des = sift.detectAndCompute(img2, None)

img1_drawKps = cv.drawKeypoints(img1, img1_kp, None)
img2_drawKps = cv.drawKeypoints(img2, img2_kp, None)

# save result
cv.imwrite('sift_keypoints.jpg',img1_drawKps)

print(img1_des.shape)
print(img1_des)

(37936, 128)
[[  1.   0.   0. ...   1.   7.   7.]
 [  3.   1.   2. ...   3.   1.   1.]
 [  0.   1.   2. ...   0.   0.   0.]
 ...
 [  1.   0.   0. ...   2.   0.   1.]
 [  0.   0.   0. ...  17.  11.  16.]
 [145.  10.   2. ...   0.   0.   0.]]


In [100]:
# Brute force matching with k=2
bf = cv.BFMatcher()
matches = bf.knnMatch(img1_des, img2_des, k=2)

# Ratio test and retrieval of indices
matches_good = [m1 for m1, m2 in matches if m1.distance < 0.75*m2.distance]

sorted_matches = sorted(matches_good, key=lambda x: x.distance)
res = cv.drawMatches(img1, img1_kp, img2, img2_kp, sorted_matches, img2, flags=2) 

cv.imwrite('sift_bfMatcher.jpg',res)
print(len(sorted_matches))

1893


In [101]:
# queryIdx : 1번 이미지의 feature point index
# trainIdx : 2번 이미지의 feature point index
query_idx = [match.queryIdx for match in matches_good]
train_idx = [match.trainIdx for match in matches_good]

p1 = np.float32([img1_kp[ind].pt for ind in query_idx])
p2 = np.float32([img2_kp[ind].pt for ind in train_idx])

# img1_pts = np.array([img1_kp[m.queryIdx].pt for m in matches_good]).reshape(-1, 1, 2).astype(np.float32) # 픽셀 좌표
# img2_pts = np.array([img2_kp[m.trainIdx].pt for m in matches_good]).reshape(-1, 1, 2).astype(np.float32)

# print(img1_pts.shape)
# print(img1_pts[:3])

In [102]:
p1

array([[  24.099752, 3219.431   ],
       [ 298.76065 , 2282.336   ],
       [ 330.07364 , 2516.3718  ],
       ...,
       [4551.96    , 3080.2961  ],
       [4573.4385  , 3094.0754  ],
       [4574.8506  , 2760.8474  ]], dtype=float32)

In [103]:
def random_five_samples(p1, p2):
    concat = np.concatenate((p1, p2), axis=1)
    rand_mat = concat[np.random.randint(concat.shape[0], size = len(concat)), :]
    p1s = rand_mat[:5, :2]
    p2s = rand_mat[:5, 2:]
    return p1s, p2s

In [104]:
def get_homogeneous(p1s, p2s, length):
    a = [[]]
    b = [[]]

    for i in range(length):
        tmp = np.append(p1s[i].flatten(), 1)
        a = np.append(a, tmp)

        tmp = np.append(p2s[i].flatten(), 1)
        b = np.append(b, tmp)

    a = a.reshape((length),3)
    b = b.reshape((length),3)

    return a,b


In [124]:
import matlab.engine

MAXITER = 100
best_inlier = 0
best_E = None
alpha = 5e-4

# intrinsic parameter
K = np.array([[3451.5, 0.0, 2312.0], [0.0, 3451.5, 1734], [0.0,0.0,1.0]])
K_inv = np.linalg.inv(K)

for _ in range(MAXITER):

    # select random points
    p1s, p2s = random_five_samples(p1, p2)
    p1s, p2s = np.array(matlab.double(p1s.tolist())), np.array(matlab.double(p2s.tolist()))
    a, b = get_homogeneous(p1s, p2s, len(p1s))

    # 5-points algorithm
    eng = matlab.engine.start_matlab()
    eng.addpath("./SfM/Step2/")  # 'calibrated_fivepoint.m'가 위치한 경로
    cur_E = np.asarray(eng.calibrated_fivepoint(a.T, b.T))
    eng.quit()

    # inlier
    for i in range(cur_E.shape[1]):
        inlier_len_temp = 0

        tmp = cur_E[:,i]
        tmp = np.reshape(tmp, [3,3])
        for j in range(5):
            estim = a[j] @ tmp @ b[j].T 
            if estim < alpha:
                inlier_len_temp += 1
        
        if best_inlier < inlier_len_temp:
            best_inlier = inlier_len_temp
            best_E = tmp

    print(tmp)
    print(best_E)

        



[[ 6.72128611e-05 -5.74084019e-04 -3.43018902e-01]
 [ 5.55371340e-04 -2.99083269e-04  6.18334720e-01]
 [-3.19838011e-01  6.30637047e-01  1.89800275e-04]]
[[ 6.72128611e-05 -5.74084019e-04 -3.43018902e-01]
 [ 5.55371340e-04 -2.99083269e-04  6.18334720e-01]
 [-3.19838011e-01  6.30637047e-01  1.89800275e-04]]
